In [8]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By # 웹사이트의 구성요소를 선택하기 위해 By 모듈을 불려옵니다
from selenium.webdriver.support.ui import WebDriverWait # 웹페이지가 전부 로드될때까지 기다리는 (Explicitly wait) 기능을 하는 모듈입니다
from webdriver_manager.chrome import ChromeDriverManager # 크롬에서 크롤링을 하기 위해, 웹 드라이버를 설치하는 모듈입니다
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as EC # 크롬의 어떤 부분의 상태를 확인하는 모듈입니다
import time # 정해진 시간만큼 기다리게 하기 위한 패키지입니다
from selenium.webdriver.common.keys import Keys

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080')#크롬드라이버 창크기
options.add_argument("headless") #창 숨기기
options.add_argument('disable-gpu') #그래픽 성능 낮춰서 크롤링 성능 높아기
options.add_argument('lang=ko_KR')
driver = webdriver.Chrome(options=options)

In [10]:
Tearchervile_url= "https://www.teacherville.co.kr/trainguide/review/reviewList.edu"

In [11]:
# driver = webdriver.Chrome()
driver.get(Tearchervile_url)

In [12]:
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]'))
    )
finally:
    pass

In [ ]:
brk = 0
nxt = 0
res = [] # 결과 파일은 판다스 데이터프레임으로 입력할겁니다
page_count = 0
stop_code = 0

while True:
    
    page_count += 1
    brk += 1

    time.sleep(1)
    hs = driver.page_source
    soup = BeautifulSoup( hs, 'html.parser')
    h = soup.find('ul',{'id':'reviewResultList'})
    
    for i in h.find_all('li'):
        date = i.find('span','day').text
        hour = i.find('strong','credit').text
        name = i.find_all('p')[1].text.split(']')[1].strip()
        cate = i.find_all('p')[1].text.split(']')[0].split('/')[1].strip()
        text = i.find('p','text on').text.strip()
        star = len(i.find_all('span','star on'))
        res.append([hour,name,text,star,cate,date])

        year = date.split('/')[0].strip()
        if int(year) <= 2020:
            stop_code += 1
            break

    if stop_code == 1:
        print(brk)
        driver.quit()
        break
    
   
    time.sleep(0.2)
    
    # 다음 페이지로 넘어가는 코드입니다    
    if page_count != 10:
        driver.find_element(By.XPATH, f'//*[@id="divPaging"]/a[{page_count+nxt}]').send_keys(Keys.ENTER)

    else:
        if brk == 10:
            driver.find_element(By.XPATH, '//*[@id="divPaging"]/a[10]').send_keys(Keys.ENTER)
            page_count = 0
            nxt += 2
            print(brk)

   
        else:
            driver.find_element(By.XPATH, '//*[@id="divPaging"]/a[12]').send_keys(Keys.ENTER)
            page_count = 0
            print(brk)

    
    time.sleep(2)



            





In [14]:
df = pd.DataFrame(res)
df.to_csv('./res_Tearchervile_final.csv', index=False) # 데이터가 실시간으로 저장되도록 합니다
df

,0,1,2,3,4,5
0,1학점,학교에서 만나는 보훈문화교육(중등),보훈의 의미를 알고 학생들을 지도하는데 많은 도움이 될것이란 생각이 듭니다.,5,학습지도,2023/10/24
1,1학점,학교에서 만나는 보훈문화교육(중등),보훈의 의미를 알고 학생들을 지도하는데 많은 도움이 될것이란 생각이 듭니다.,5,학습지도,2023/10/24
2,0학점,"융합교육의 완성, 전문가가 알려주는 ChatGPT",학교의 컴퓨터 환경에서는 챗 기능을 활용하기 어려운가 봅니다.,5,디지털활용,2023/10/24
3,1학점,주식의 정석(실전편)(직무상시),궁굼했던 부분이 잘 해결되었고 잘 들었습니다. 또 기회되면 들어볼께요. 수고하세요,5,인문·교양,2023/10/24
4,0학점,노션(Notion)으로 만들고 활용하는 디지털 교무수첩,유익한 교육이였습니다.\n향후 노션 반복활용으로 효율적인 적용이 될 수 있도록 하고...,5,디지털활용,2023/10/24
...,...,...,...,...,...,...
54049,4학점,위로와 격려로 다시 쓰는 교사 위기 극복 가이드,위로받는 느낌으로 연수를 받았습니다. 연수 내내 지루하지 않았어요. 선생님들 힘내세요.,5,생활지도,2022/12/05
54050,2학점,"돈으로 움직이는 교실 이야기, 세금 내는 아이들(부산교육연수원)",실생활에 너무 도움이되는 유익한 연수였습니다 감사합니다,5,학습지도,2022/12/05
54051,2학점,정재승의 스쿨 브레인,창의성의 개념을 뇌과학적으로 바라볼 수 있게 해준다는 점에서 신선한 강의. 다만 뇌...,5,인문·교양,2022/12/05
54052,0학점,야! 나도 온라인으로 수업할 수 있어!,좋은 강의 감사합니다. 강의 보면서 여러가지 수업 방법을 도입 해야 겠다는 생각을 ...,5,디지털활용,2022/12/05
